In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
data = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/CT_metdata_copy_1.csv')
journal_meta = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/journal_metadata.csv')

In [3]:
journal_meta

,issn,name,h_index,publication_type,sjr_rank,sjr_quartile
0,1364-8535,Critical Care,185.0,Journals,3.218,Q1
1,1465-993X,Respiratory Research,114.0,Journals,1.813,Q1
2,1364-8535,Critical Care,185.0,Journals,3.218,Q1
3,1465-993X,Respiratory Research,114.0,Journals,1.813,Q1
4,1471-2458,BMC Public Health,159.0,Journals,1.156,Q1
...,...,...,...,...,...,...
1215,0100-879X,Brazilian Journal of Medical and Biological Re...,91.0,Journals,0.522,Q3
1216,0350-199X,Medicinski Arhiv,26.0,Journals,0.294,Q3
1217,0077-8923,Annals of the New York Academy of Sciences,261.0,Journals,1.520,Q1
1218,2398-0648,NaN,NaN,NaN,NaN,NaN


In [4]:
journal_meta['publication_type'].unique()

array(['Journals', nan, 'Book Series'], dtype=object)

In [15]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'cord_uid', 'sha', 'source_x', 'title',
       'doi', 'pmcid', 'pubmed_id', 'license', 'abstract', 'publish_time',
       'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id',
       'pdf_json_files', 'pmc_json_files', 'url', 's2_id', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'doi_replace', 'abstract_replace', 'crossref_title',
       'crossref_abstract', 'crossref_journal', 'crossref_authors',
       'crossref_publish_time', 'issn', 'crossref_publication_type'],
      dtype='object')

In [5]:
issn = data['issn'].dropna().str.replace("[\[\]']", '').str.split(",").to_list()

/tmp/ipykernel_78363/2596356645.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  issn = data['issn'].dropna().str.replace("[\[\]']", '').str.split(",").to_list()


In [6]:
max(issn,key=len)

['1741-427X', ' 1741-4288']

In [62]:
for i in range(len(issn)):
    for j in range(len(issn[i])):
        print(issn[i][j])
        issn[i][j] = issn[i][j].strip()
        print(issn[i][j])
        print("----------------------------------------")

1364-8535
1364-8535
----------------------------------------
1364-8535
1364-8535
----------------------------------------
1741-427X
1741-427X
----------------------------------------
 1741-4288
1741-4288
----------------------------------------
1465-993X
1465-993X
----------------------------------------
1471-2458
1471-2458
----------------------------------------
1476-0711
1476-0711
----------------------------------------
1364-8535
1364-8535
----------------------------------------
1932-6203
1932-6203
----------------------------------------
1364-8535
1364-8535
----------------------------------------
1364-8535
1364-8535
----------------------------------------
0305-1048
0305-1048
----------------------------------------
 1362-4962
1362-4962
----------------------------------------
1932-6203
1932-6203
----------------------------------------
1750-1172
1750-1172
----------------------------------------
1932-6203
1932-6203
----------------------------------------
1364-8535
1364-8535
--

In [72]:
issn

[['1364-8535'],
 ['1364-8535'],
 ['1741-427X', '1741-4288'],
 ['1465-993X'],
 ['1471-2458'],
 ['1476-0711'],
 ['1364-8535'],
 ['1932-6203'],
 ['1364-8535'],
 ['1364-8535'],
 ['0305-1048', '1362-4962'],
 ['1932-6203'],
 ['1750-1172'],
 ['1932-6203'],
 ['1364-8535'],
 ['1863-2297', '1863-2300'],
 ['1381-6128'],
 ['1364-8535'],
 ['0928-4249', '1297-9716'],
 ['1932-6203'],
 ['1110-7243', '1110-7251'],
 ['1080-6040', '1080-6059'],
 ['1741-427X', '1741-4288'],
 ['1932-6203'],
 ['0031-3998', '1530-0447'],
 ['1932-6203'],
 ['1750-1172'],
 ['1932-6203'],
 ['1549-1676'],
 ['1932-6203'],
 ['1549-1676'],
 ['0304-8608', '1432-8798'],
 ['1935-2735'],
 ['0305-1048', '1362-4962'],
 ['1932-6203'],
 ['2090-1364', '2090-1372'],
 ['2042-6410'],
 ['1471-2466'],
 ['1465-993X'],
 ['1471-2172'],
 ['1743-422X'],
 ['1553-7358'],
 ['1076-1551', '1528-3658'],
 ['1742-5689', '1742-5662'],
 ['1932-6203'],
 ['1471-2334'],
 ['2090-3014', '2090-3022'],
 ['1745-6215'],
 ['1058-4838', '1537-6591'],
 ['1932-6203'],
 ['15

In [76]:
issn_done = []

In [77]:
def html_parse(issn):
    sjr_base_url = f'https://www.scimagojr.com/journalsearch.php?q='
    # Set the headers with user-agent
    headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
                }
    url = sjr_base_url+issn
    response = requests.get(url,headers=headers)

    return BeautifulSoup(response.content, 'html.parser'),issn

def extract_info(soup,issn_):
    global journal_meta, issn_done
    search_div = soup.find('div', class_='search_results')
    # Journal Name
    journal_name = search_div.find('span',class_= 'jrnlname').get_text()

    res = requests.get('https://www.scimagojr.com/'+str(search_div.find('a').get('href')))
    print(res.status_code)
    soup_div = BeautifulSoup(res.content, 'html.parser')
    
    journal_grid = soup_div.find('div', class_='journalgrid').find_all('div')
    # h-index
    h_index = journal_grid[3].p.get_text()
    # h-index
    publication_type = journal_grid[4].p.get_text()
    
    dashboard = soup_div.find_all('div',class_='dashboard')
    # SJR Rank
    sjr_rank = dashboard[1].find('div').find('div',class_='cellcontent').find('table').find('tbody').find_all('tr')[-1].find_all('td')[-1].get_text()
    # SJR Quartile(Q1,Q2,Q3,Q4)
    sjr_quartile = dashboard[0].find('div').find('div',class_='cellcontent').find('table').find('tbody').find_all('tr')[-1].find_all('td')[-1].get_text()
    
    # print(res.status_code)
    # print("ISSN ",issn)
    print("Journal: {}\n H-index: {}\n Publication type: {}\n Rank: {}\n Quartile: {}".format(journal_name,h_index,publication_type,sjr_rank,sjr_quartile))
    print('-------------------------------')
    journal_meta = journal_meta.append({'issn':issn_, 'name':journal_name, 'h_index':h_index, 'publication_type':publication_type,
                        'sjr_rank':sjr_rank, 'sjr_quartile':sjr_quartile },ignore_index = True)
    journal_meta.to_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/journal_metadata.csv',index=False)
    issn_done.append(issn_)

In [ ]:
for i in tqdm(range(len(issn))):
    # for 2 issns
    if len(issn[i]) ==2:
        print(issn[i][0],'\t',issn[i][1])  
        if issn[i][0] not in issn_done and issn[i][1] not in issn_done:
            soup,issn_  = html_parse(issn[i][0])
            # If issn is valid or not
            if not soup.find('div', class_='search_results').find('span'):
                soup,issn_ = html_parse(issn[i][1])
                if soup.find('div', class_='search_results').find('span'):
                    print("ACTUAL ",issn_)
                    extract_info(soup,issn_)
                else:
                    journal_meta = journal_meta.append({'issn':issn_, 'name':'NA', 'h_index':'NA', 
                                                        'publication_type':'NA','sjr_rank':'NA', 'sjr_quartile':'NA' },ignore_index = True)
                    journal_meta.to_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/journal_metadata.csv',index=False)
                    issn_done.append(issn_)
    # for single issn
    else:
        print(issn[i][0])
        if issn[i][0] not in issn_done:
            soup,issn_ = html_parse(issn[i][0])
            if soup.find('div', class_='search_results').find('span'):
                extract_info(soup,issn_)
            else:
                journal_meta = journal_meta.append({'issn':issn_, 'name':'NA', 'h_index':'NA', 
                                                    'publication_type':'NA','sjr_rank':'NA', 'sjr_quartile':'NA' },ignore_index = True)
                journal_meta.to_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/journal_metadata.csv',index=False)
                issn_done.append(issn_)
    


In [52]:
issn_done

['1364-8535',
 '1465-993X',
 '1471-2458',
 '1476-0711',
 '1932-6203',
 '1750-1172',
 '1381-6128']